# <p style="text-align:center">Travail personel 4: Équations différentielles partielles</p> #

**Mathieu Dumais**, **Antoine Rousseau** et **Gabriel Lapointe**.

Présenté à **Philippe Després**.

Date: **4 avril 2021**.

In [ ]:
from scipy.linalg import solve_banded
from numpy import matmul, exp, linspace, array, zeros, empty
from pylab import plot, show, ylabel, xlabel

# Question 1


# Question 2


# Question 3


# Question 4


# Question 5
